# Testing Agents v2 in AI Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

## Step 1
Create Project client

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import atexit
import jsonref
import os
from azure.identity.aio import DefaultAzureCredential, AzureDeveloperCliCredential
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import (
    MCPTool,
    Tool,
)
from openai.types.responses.response_input_param import (
    McpApprovalResponse,
    ResponseInputParam,
)
from dotenv import load_dotenv
from src.agents_utils import agents_utils


# Load environment variables from the .env file
load_dotenv(override=True)

print("🚀 Initializing AI Foundry Agent Testing...")
print("=" * 60)

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

print("\n📋 Configuration:")
print(f"   📍 Endpoint: {endpoint[:50]}..." if endpoint else "   ⚠️ Endpoint: Not set")
print(f"   🤖 Model: {deployment_name}" if deployment_name else "   ⚠️ Model: Not set")
print(f"   📦 API Version: {api_version or 'Default'}")

ai_agent_settings = {
    "endpoint": endpoint,
    "model_deployment_name": deployment_name,
    "api_version": api_version,
}

# Setup credentials
print("\n🔐 Setting up authentication...")
if os.environ.get("USE_AZURE_DEV_CLI") == "true":
    print("   ✅ Using Azure Developer CLI Credential")
    creds = AzureDeveloperCliCredential(tenant_id=tenant_id)
else:
    print("   ✅ Using Default Azure Credential")
    creds = DefaultAzureCredential()

await creds.__aenter__()
print("   🔓 Credentials initialized successfully!")

# Initialize clients
print("\n🔧 Initializing clients...")
client = AIProjectClient(endpoint=endpoint, credential=creds)
await client.__aenter__()
agents_client = agents_utils(client)
print("   ✅ AI Project Client ready")
print("   ✅ Agents Client ready")

# List connections
print("\n" + "=" * 60)
print("🔗 AVAILABLE CONNECTIONS")
print("=" * 60)

model_gateway_connection_static = None
model_gateway_connection_dynamic = None
ai_gateway_connection_static = None
ai_gateway_connection_dynamic = None

connection_count = 0
async for connection in client.connections.list():
    connection_count += 1
    icon = (
        "🌐"
        if connection.type == "ModelGateway"
        else "🔌" if connection.type == "ApiManagement" else "📡"
    )
    default_badge = " ⭐ DEFAULT" if connection.is_default else ""
    print(f"\n{icon} {connection.name}{default_badge}")
    print(f"   Type: {connection.type}")
    print(f"   ID: {connection.id[:50]}...")

    if connection.type == "ModelGateway" and "static" in connection.name.lower():
        model_gateway_connection_static = connection.name
        print("   📌 → Assigned to: model_gateway_connection_static")
    if connection.type == "ModelGateway" and "static" not in connection.name.lower():
        model_gateway_connection_dynamic = connection.name
        print("   📌 → Assigned to: model_gateway_connection_dynamic")
    if connection.type == "ApiManagement" and "static" in connection.name.lower():
        ai_gateway_connection_static = connection.name
        print("   📌 → Assigned to: ai_gateway_connection_static")
    if connection.type == "ApiManagement" and "static" not in connection.name.lower():
        ai_gateway_connection_dynamic = connection.name
        print("   📌 → Assigned to: ai_gateway_connection_dynamic")

print(f"\n📊 Total connections found: {connection_count}")

# List agents
print("\n" + "=" * 60)
print("🤖 EXISTING AGENTS")
print("=" * 60)

agents = await agents_client.get_agents()
if agents:
    for agent in agents:
        print(f"\n🤖 {agent.name}")
        print(f"   ID: {agent.id}")
        print(f"   Version: {agent.versions.latest.version}")
else:
    print("\n   ℹ️ No agents found in this project")

# Summary
print("\n" + "=" * 60)
print("✅ SETUP COMPLETE - Connection Summary")
print("=" * 60)
print(
    f"   🌐 Model Gateway (Static):  {'✅ ' + model_gateway_connection_static if model_gateway_connection_static else '❌ Not found'}"
)
print(
    f"   🌐 Model Gateway (Dynamic): {'✅ ' + model_gateway_connection_dynamic if model_gateway_connection_dynamic else '❌ Not found'}"
)
print(
    f"   🔌 AI Gateway (Static):     {'✅ ' + ai_gateway_connection_static if ai_gateway_connection_static else '❌ Not found'}"
)
print(
    f"   🔌 AI Gateway (Dynamic):    {'✅ ' + ai_gateway_connection_dynamic if ai_gateway_connection_dynamic else '❌ Not found'}"
)
print("\n🎉 Ready to run agent tests!")


async def cleanup():
    """Close all async clients properly"""
    try:
        await client.close()
    except Exception:
        await client.__aexit__(None, None, None)
    try:
        await creds.close()
    except Exception:
        await creds.__aexit__(None, None, None)
    print("🧹 Clients closed")


def sync_cleanup():
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(cleanup())
        else:
            loop.run_until_complete(cleanup())
    except Exception:
        print("⚠️ Cleanup failed")
        pass


atexit.register(sync_cleanup)

## Run Tests

In [ ]:
# Run multiple requests for each gateway connection and capture request IDs
import pandas as pd
from openai import APIStatusError, APIError
from azure.core.exceptions import HttpResponseError

print("🧪 GATEWAY CONNECTION TESTS")
print("=" * 60)
print("Testing each gateway connection with multiple requests to verify")
print("connectivity and capture request IDs for debugging.\n")

# Collect all gateway connections to test
gateway_connections = {
    "model_gateway_static": model_gateway_connection_static,
    "model_gateway_dynamic": model_gateway_connection_dynamic,
    "ai_gateway_static": ai_gateway_connection_static,
    "ai_gateway_dynamic": ai_gateway_connection_dynamic,
}
deployment_model: str = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Filter out None connections
active_connections = {k: v for k, v in gateway_connections.items() if v is not None}
print(f"📊 Found {len(active_connections)} active connections to test:")
for name in active_connections.keys():
    print(f"   • {name}")

results = []
num_requests = 5
delete_agent_before_create = False

print(f"\n⚙️ Test Configuration:")
print(f"   • Requests per connection: {num_requests}")
print(f"   • Model: {deployment_model}")
print(f"   • Delete agent before create: {delete_agent_before_create}")

for conn_name, conn_value in active_connections.items():
    print(f"\n{'─' * 60}")
    print(f"🔄 Testing: {conn_name}")
    print(f"   Connection: {conn_value}")
    print(f"{'─' * 60}")

    for i in range(num_requests):
        request_id = None
        result = None
        error_msg = None

        try:
            agent_name = f"TestAgent_{conn_name}".replace(" ", "-").replace("_", "-")[
                :63
            ]

            # Create agent for this connection
            print(f"   📝 Request {i+1}/{num_requests}: Creating agent...", end=" ")
            agent = await agents_client.create_agent(
                name=agent_name,
                model_gateway_connection=conn_value,
                deployment_name=deployment_model,
                delete_before_create=delete_agent_before_create,
            )

            openai_client = client.get_openai_client()

            # Create conversation
            conversation = await openai_client.conversations.create(
                items=[
                    {
                        "type": "message",
                        "role": "user",
                        "content": f"What is 2 + {i}? Reply with just the number.",
                    }
                ],
            )

            # Get response
            response = await openai_client.responses.create(
                conversation=conversation.id,
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
                input="",
            )

            # Extract request ID from response
            request_id = response._request_id
            result = "SUCCESS"
            output = response.output_text[:50] if response.output_text else "No output"

            results.append(
                {
                    "Connection": conn_name,
                    "Model": deployment_model,
                    "Request #": i + 1,
                    "Request ID": request_id,
                    "Result": result,
                    "Output": output,
                }
            )
            print(f"✅ SUCCESS")
            print(f"      Response: {output}")
            print(f"      Request ID: {request_id}")

        except (APIStatusError, APIError, HttpResponseError) as e:
            request_id = (
                e.response.headers.get("x-request-id", "N/A")
                if hasattr(e, "response")
                else "N/A"
            )
            result = "ERROR"
            error_msg = str(e)

            results.append(
                {
                    "Connection": conn_name,
                    "Model": deployment_model,
                    "Request #": i + 1,
                    "Request ID": request_id,
                    "Result": result,
                    "Output": error_msg,
                }
            )
            print(f"❌ ERROR")
            print(f"      {error_msg}")
            print(f"      Request ID: {request_id}")

        except Exception as e:
            result = "ERROR"
            error_msg = str(e)

            results.append(
                {
                    "Connection": conn_name,
                    "Model": deployment_model,
                    "Request #": i + 1,
                    "Request ID": "N/A",
                    "Result": result,
                    "Output": error_msg,
                }
            )
            print(f"❌ ERROR")
            print(f"      {error_msg}")

# Create DataFrame and display results
df = pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 RESULTS TABLE")
print("=" * 60)
print(df.to_string(index=False))

# Save results to CSV
csv_file = "gateway_connection_test_results.csv"
df.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")

# Summary statistics
print("\n" + "=" * 60)
print("📈 SUMMARY")
print("=" * 60)
summary = df.groupby(["Connection", "Result"]).size().unstack(fill_value=0)
print(summary)

# Calculate success rate
total = len(results)
successes = len([r for r in results if r["Result"] == "SUCCESS"])
success_rate = (successes / total * 100) if total > 0 else 0
print(f"\n🎯 Overall Success Rate: {successes}/{total} ({success_rate:.1f}%)")

if success_rate == 100:
    print("🎉 All tests passed!")
elif success_rate >= 50:
    print("⚠️ Some tests failed - check the results above")
else:
    print("❌ Most tests failed - investigate connection issues")

## Call Foundry directly

In [ ]:
print("🔗 DIRECT FOUNDRY CALL")
print("=" * 60)
print("Calling AI Foundry directly without agent abstraction")
print(f"   📍 Connection: {model_gateway_connection_static}")
print(f"   🤖 Model: {deployment_name}")
print()

openai_client = client.get_openai_client()

response = await openai_client.responses.create(
    model=f"{model_gateway_connection_static}/{deployment_name}",
    input="I am going to Paris, what should I see? Keep your answer brief.",
    instructions="You are a helpful assistant.",
)

print("✅ Response received!")
print(f"   🆔 Request ID: {response._request_id}")
print("\n💬 Response:")
print("─" * 40)
print(response.output_text)
print("─" * 40)

## Run agent using static gateway

In [ ]:
print("🤖 AGENT WITH STATIC GATEWAY")
print("=" * 60)
print(f"   🔗 Connection: {ai_gateway_connection_static}")
print()

print("📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MyV2Agent",
    model_gateway_connection=ai_gateway_connection_static,
    delete_before_create=True,
)
print(f"   ✅ Agent created: {agent.name}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "What is the size of Poland in square miles?",
        }
    ],
)
print(f"   💬 Conversation started: {conversation.id[:20]}...")

print("\n⏳ Waiting for response...")
response = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)

print("\n✅ Response received!")
print(f"   🆔 Response ID: {response.id}")
print("\n💬 Output:")
print("─" * 40)
print(response.output_text)
print("─" * 40)
print(f"\n💰 Usage: {response.to_dict()['usage']}")

## Run agent using dynamic gateway

In [ ]:
print("🤖 AGENT WITH DYNAMIC GATEWAY")
print("=" * 60)
print(f"   🔗 Connection: {model_gateway_connection_dynamic}")
print()

print("📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MyV2Agent",
    model_gateway_connection=model_gateway_connection_dynamic,
    delete_before_create=True,
)
print(f"   ✅ Agent created: {agent.name}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "What is the history of Warsaw? Keep it brief.",
        }
    ],
)
print(f"   💬 Conversation started: {conversation.id[:20]}...")

print("\n⏳ Waiting for response...")
response = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)

print("\n✅ Response received!")
print("\n💬 Output:")
print("─" * 40)
print(response.output_text)
print("─" * 40)
print(f"\n💰 Usage: {response.to_dict()['usage']}")

## Streaming

In [ ]:
print("🌊 STREAMING RESPONSE TEST")
print("=" * 60)
print("Testing real-time streaming of agent responses")
print()

print("📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MyAgentGpt5Mini",
    model_gateway_connection=model_gateway_connection_dynamic,
    delete_before_create=True,
)
print(f"   ✅ Agent: {agent.name}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "Tell me hi in 10 random languages.",
        }
    ],
)
print(f"   💬 Conversation: {conversation.id[:20]}...")

# Create streaming response
response_stream_events = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
    stream=True,
)

print("\n🌊 Streaming response:")
print("─" * 40)
events_received = []

async for event in response_stream_events:
    previous_event = events_received[-1] if len(events_received) > 0 else None
    if previous_event and previous_event["type"] == event.type:
        previous_event["count"] += 1
    else:
        events_received.append({"type": event.type, "count": 1})

    if event.type == "response.created":
        print(f"🆕 Stream started (ID: {event.response.id})\n")
    elif event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    elif event.type == "response.text.done":
        print("\n" + "─" * 40)
        print("✅ Text complete")
    elif event.type == "response.completed":
        print(f"\n🎉 Response completed!")
        print(f"💰 Usage: {event.response.to_dict()['usage']}")

print("\n📊 Events received:")
for index, event in enumerate(events_received):
    print(f"   {index+1}. {event['type']} (×{event['count']})")

## Tool Calls

In [ ]:
from src.agents_utils import process_stream
from openai import NOT_GIVEN

print("🔧 STREAMING MCP TOOL CALLS")
print("=" * 60)
print("Testing MCP tool integration with streaming responses")
print()

# Configure MCP tool
mcp_tool = MCPTool(
    server_label="ms-learn",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="always",
)
print("🔌 MCP Tool configured:")
print(f"   Label: {mcp_tool.server_label}")
print(f"   URL: {mcp_tool.server_url}")
print(f"   Approval: {mcp_tool.require_approval}")

tools: list[Tool] = [mcp_tool]

print("\n📝 Creating agent with tools...")
agent = await agents_client.create_agent(
    name="MyAgentGpt5MiniTools",
    model_gateway_connection=model_gateway_connection_static,
    delete_before_create=True,
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=tools,
)
print(f"   ✅ Agent: {agent.name}")
print(f"   🤖 Model: {agent.versions.latest.definition.model}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "Find me learning paths on Microsoft Learn about Azure AI services.",
        }
    ],
)
print(f"   💬 Conversation: {conversation.id[:20]}...")

# Streaming loop with tool approval
events_received = []
input_list = []
response_id = None
input = ""
request_count = 0

print("\n🔄 Starting tool call loop...")
print("─" * 60)

while True:
    response_stream_events = await openai_client.responses.create(
        conversation=conversation.id if response_id is None else NOT_GIVEN,
        previous_response_id=response_id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        input=input,
        stream=True,
    )

    print(f"\n🌊 Stream iteration #{request_count + 1}:")
    events_received, input_list, response_id, full_response = await process_stream(
        response_stream_events
    )

    print(f"\n📊 Events in this iteration:")
    for index, event in enumerate(events_received):
        print(f"   {index+1}. {event['type']} (×{event['count']})")

    if len(input_list) == 0:
        break

    input = input_list
    print(f"\n✅ Auto-approving {len(input_list)} tool request(s)...")
    request_count += 1

print("\n" + "=" * 60)
print("🎉 Tool call loop completed!")
print(f"   Total iterations: {request_count + 1}")

## Non Streaming MCP

In [ ]:
from src.agents_utils import process_response

print("🔧 NON-STREAMING MCP TOOL CALLS")
print("=" * 60)
print("Testing MCP tools without streaming (synchronous)")
print()

# Configure MCP tool
mcp_tool = MCPTool(
    server_label="docs",
    server_url="https://gitmcp.io/Azure/azure-rest-api-specs",
    require_approval="always",
)
print(f"🔌 MCP Tool configured:")
print(f"   Label: {mcp_tool.server_label}")
print(f"   URL: {mcp_tool.server_url}")

# Load OpenAPI spec (optional - not used in this example)
with open("weather.json", "r") as f:
    openapi_weather = jsonref.loads(f.read())
    openapi_weather["servers"] = [{"url": "https://wttr.in"}]
print(f"🌤️ Weather OpenAPI spec loaded (available if needed)")

tools: list[Tool] = [mcp_tool]

print("\n📝 Creating agent with tools...")
agent = await agents_client.create_agent(
    name="MyAgentGpt5MiniTools",
    model_gateway_connection=model_gateway_connection_static,
    delete_before_create=True,
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=tools,
)
print(f"   ✅ Agent: {agent.name}")

openai_client = client.get_openai_client()
response_id = None
input = ""
input_list: ResponseInputParam = []
request_count = 0
conversation = await openai_client.conversations.create(
    items=[
        {"type": "message", "role": "user", "content": "Summarize the readme from attached MCP tool for me. Keep it very brief."}
    ],
)
print(f"   💬 Conversation: {conversation.id[:20]}...")

print("\n⏳ Sending initial request...")
while True:
    response = await openai_client.responses.create(
        conversation=conversation.id if response_id is None else NOT_GIVEN,
        previous_response_id=response_id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        input=input,
        stream=False,
    )

    print(f"\n🌊 Stream iteration #{request_count + 1}:")
    events_received, input_list, response_id, full_response = await process_response(
        response
    )

    print("\n📊 Events in this iteration:")
    for index, event in enumerate(events_received):
        print(f"   {index+1}. {event['type']} (×{event['count']})")

    if len(input_list) == 0:
        break

    input = input_list
    print(f"\n✅ Auto-approving {len(input_list)} tool request(s)...")
    request_count += 1

response = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)

print("\n" + "=" * 60)
print("✅ RESULT")
print("=" * 60)
print(response.output_text)
print("\n💰 Full response details:")
print(f"   Usage: {response.to_dict().get('usage', 'N/A')}")

## 🔧 Agent MCP Loop Tests
Run multiple requests using an agent with MCP tools and capture:
- ✅ Success/Error status for each iteration
- 🆔 Request IDs for debugging
- 📊 Summary statistics and success rate
- 💾 Results exported to CSV

In [ ]:
import pandas as pd
from openai import APIStatusError, APIError, NOT_GIVEN
from azure.core.exceptions import HttpResponseError
from src.agents_utils import process_response


print("🔧 MCP WITH AGENT - LOOP TEST")
print("=" * 60)
print("Running MCP tools with agent abstraction")
print("Testing with 10 iterations to capture errors and request IDs")
print()

# Configure MCP tool
mcp_tool = MCPTool(
    server_label="weather",
    server_url="https://aca-mcp-qczp34j2qg7pk.ashyocean-7ea49412.westus.azurecontainerapps.io/mcp/mcp",
    require_approval="never",
)
print(f"🔌 MCP Tool: {mcp_tool.server_label}")
print(f"   URL: {mcp_tool.server_url}")

tools: list[Tool] = [mcp_tool]

# Create agent once at the top
print("\n📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MCPLoopTestAgent",
    # model_gateway_connection=model_gateway_connection_static,
    deployment_name="gpt-5",
    delete_before_create=True,
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=tools,
)
print(f"   ✅ Agent: {agent.name}")
print(f"   🤖 Model: {agent.versions.latest.definition.model}")

openai_client = client.get_openai_client()

# Configuration
num_iterations = 10
results = []

print(f"\n⚙️ Test Configuration:")
print(f"   • Iterations: {num_iterations}")
print(f"   • Agent: {agent.name}")
print()

for iteration in range(num_iterations):
    print(f"\n{'─' * 60}")
    print(f"🔄 Iteration {iteration + 1}/{num_iterations}")
    print(f"{'─' * 60}")
    
    request_id = None
    result_status = None
    error_msg = None
    output_text = None
    
    try:
        # Create new conversation for each iteration
        random_number_a = iteration * 7 + 3
        random_number_b = iteration * 11 + 5
        conversation = await openai_client.conversations.create(
            items=[
                {"type": "message", "role": "user", "content": f"Add {random_number_a} and {random_number_b} using MCP tool."}
            ],
        )
        print(f"   💬 Conversation: {conversation.id[:20]}...")
        response_id = None
        input = ""
        input_list: ResponseInputParam = []
        request_count = 0

        while True:
            print("   ⏳ Sending request...")
            response = await openai_client.responses.create(
                conversation=conversation.id if response_id is None else NOT_GIVEN,
                previous_response_id=response_id,
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
                input=input,
            )

            print(f"\n🌊 Request iteration #{request_count + 1}:")
            events_received, input_list, response_id, full_response = await process_response(
                response
            )

            if len(input_list) == 0:
                break

            input = input_list
            print(f"\n✅ Auto-approving {len(input_list)} tool request(s)...")
            request_count += 1


        request_id = response._request_id

        result_status = "SUCCESS"
        output_text = response.output_text[:100] if response.output_text else "No output"
        print(f"   ✅ SUCCESS")
        print(f"      Request ID: {request_id}")
        print(f"      Output: {output_text[:50]}...")

    except (APIStatusError, APIError, HttpResponseError) as e:
        request_id = (
            e.response.headers.get("x-request-id", "N/A")
            if hasattr(e, "response") and e.response is not None
            else "N/A"
        )
        result_status = "ERROR"
        error_msg = str(e)
        print(f"   ❌ ERROR")
        print(f"      Request ID: {request_id}")
        print(f"      Error: {error_msg}")

    except Exception as e:
        result_status = "ERROR"
        error_msg = str(e)
        print(f"   ❌ ERROR")
        print(f"      Request ID: N/A")
        print(f"      Error: {error_msg}")

    # Record results
    results.append({
        "Iteration": iteration + 1,
        "Request ID": request_id or "N/A",
        "Result": result_status,
        "Output/Error": output_text if result_status == "SUCCESS" else error_msg,
    })

# Create DataFrame and display results
df_results = pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 RESULTS TABLE")
print("=" * 60)
print(df_results.to_string(index=False))

# Save results to CSV
csv_file = "mcp_agent_test_results.csv"
df_results.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")

# Summary statistics
print("\n" + "=" * 60)
print("📈 SUMMARY")
print("=" * 60)
total = len(results)
successes = len([r for r in results if r["Result"] == "SUCCESS"])
errors = len([r for r in results if r["Result"] == "ERROR"])
success_rate = (successes / total * 100) if total > 0 else 0

print(f"   ✅ Successes: {successes}")
print(f"   ❌ Errors: {errors}")
print(f"   🎯 Success Rate: {success_rate:.1f}%")

if success_rate == 100:
    print("\n🎉 All tests passed!")
elif success_rate >= 50:
    print("\n⚠️ Some tests failed - check the results above")
else:
    print("\n❌ Most tests failed - investigate connection issues")

# Display unique request IDs for debugging
print("\n📋 Request IDs:")
for r in results:
    status_icon = "✅" if r["Result"] == "SUCCESS" else "❌"
    print(f"   {status_icon} #{r['Iteration']}: {r['Request ID']}")